In [1]:
import numpy as np
import numpy.linalg as alg
import math
import scipy.optimize as opt
import copy
import pandas as pd

In [2]:
sup10 = np.load('data/sup10.npy')
sup11 = np.load('data/sup11.npy')

In [3]:
A = sup10
B = sup11

In [4]:
u = np.sum(B, axis=1)  # Summ by rows
v = np.sum(B, axis=0)  # Summ by columns

## 2.3 RAS

Please refer to the "RAS_article" presented in out repository for more details about this method

In [5]:
def diog(x):
    return np.eye(x.shape[0]) * x

In [7]:
def RAS_pred(previous_year, current_year, eps):
    A = previous_year
    A = A + 10**(-8)  # We need to add some noise in order to avoid mistakes computing everse matrix
    B = current_year
    u = np.sum(B, axis=1)
    v = np.sum(B, axis=0)
    r = np.ones(A.shape[0])
    s = np.ones(A.shape[1])
    while 1:
        matrix1 = diog((A.dot(alg.inv(diog(A.T.dot(r))))).dot(v))
        matrix2 = diog((A.T.dot(alg.inv(diog(A.dot(s))))).dot(u))
        new_r = alg.inv(matrix1).dot(u)
        new_s = alg.inv(matrix2).dot(v)
        if alg.norm(new_r - r) < eps and alg.norm(new_s - s) < eps:
            r = (alg.inv(diog(A.dot(new_s)))).dot(u)
            s = (alg.inv(diog(A.T.dot(new_r)))).dot(v)
            break
        r = new_r
        s = new_s
    return (diog(r).dot(A)).dot(diog(s))

# All the Following methods are taken from the "MainArticle" devoted to our project theme.

## 2.4 Improved Normalized squared difference (INS)

In [8]:
def next_lambda(lambd, tau, z, A, u, M):
    min_z = np.where(z < 0, z, 0)
    result = lambd.copy()
    for i in range(result.shape[0]):
        expr = A[i]*min_z[i]*M - tau*np.abs(A[i])
        denominator = np.sum(np.abs(A[i]))
        if denominator == 0:
            result[i] = 0
        else:
            result[i] = (u[i] - np.sum(A[i]) + np.sum(expr)) / denominator
    return result

In [9]:
def next_tau(lambd, tau, z, A, v, M):
    min_z = np.where(z.T < 0, z.T, 0)
    result = tau.copy()
    for j in range(result.shape[0]):
        expr = A.T[j]*min_z[j]*M - lambd*np.abs(A.T[j])
        denominator = np.sum(np.abs(A.T[j]))
        if denominator == 0:
            result[j] = 0
        else:
            result[j] = (v[j] - np.sum(A.T[j]) + np.sum(expr)) / denominator
    return result

In [10]:
def compute_z(lambd, tau, z, A, M):
    result = copy.deepcopy(z)
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            if A[i][j] == 0:
                result[i][j] = 1
            else:
                var = 1 + ((lambd[i] + tau[j])*A[i][j] / abs(A[i][j]))
                if var >= 0:
                    result[i][j] = var
                else:
                    result[i][j] = var / (1 + M)
    return result

In [11]:
M = 1000  # Hyperparameter desired to be as big as possible.  
z = np.ones(A.shape)
lambd = np.zeros(u.shape[0])
tau = np.zeros(v.shape[0])
eps = 1e-9  # The desired accuracy.
while 1:
    new_lambd = next_lambda(lambd, tau, z, A, u, M)
    new_tau = next_tau(new_lambd, tau, z, A, v, M)
    z = compute_z(new_lambd, new_tau, z, A, M)
    if np.linalg.norm(new_lambd - lambd) < eps and np.linalg.norm(new_tau - tau) < eps:
        break
    lambd = new_lambd
    tau = new_tau

In [12]:
np.sum(np.round(z*A), axis=0)  # Summ by rows of our predicted matrix

array([27693.,   244.,   512., 22399., 57960.,  3498.,  2704.,  5990.,
        4687., 32868., 47939.,  5497.,  7557.,  5964.,  8800., 18640.,
       27719.,  5668., 25233.,  7289.,  6039., 11105.,  8256., 18539.,
        1674.,  7806., 91130., 16979., 83954., 35687., 27409.,  7307.,
        9679., 18862.,  5027., 20049.,  6182.,  4313., 17056., 24503.,
       48632., 21409.,  5965., 69809., 52935., 15011.,  4394.,  8097.,
        6078., 10143., 19097.,  7381., 14314., 74460., 37910., 41940.,
       33790.,  8463.,  5703.,  6053.,   834.,  5120.,   445.,     0.])

In [13]:
np.sum(B, axis=0)  # Summ by rows of actuall matrix

array([27694.,   243.,   512., 22399., 57961.,  3499.,  2703.,  5992.,
        4688., 32869., 47941.,  5497.,  7557.,  5966.,  8799., 18641.,
       27719.,  5671., 25231.,  7289.,  6038., 11103.,  8255., 18540.,
        1673.,  7806., 91131., 16981., 83954., 35689., 27409.,  7308.,
        9679., 18862.,  5027., 20050.,  6183.,  4313., 17057., 24503.,
       48631., 21409.,  5964., 69809., 52932., 15010.,  4394.,  8098.,
        6077., 10141., 19099.,  7382., 14314., 74463., 37910., 41938.,
       33790.,  8464.,  5704.,  6053.,   835.,  5120.,   445.,     0.])

## 2.5 Improved square differences (ISD)

In [14]:
def ISDnext_lambda(lambd, tau, z, A, u, M):
    min_z = np.where(z < 0, z, 0)
    result = lambd.copy()
    delta_A = np.where(A != 0, 1, 0)
    for i in range(result.shape[0]):
        expr = A[i]*min_z[i]*M - tau*delta_A[i]
        denominator = np.sum(delta_A[i])
        if denominator == 0:
            result[i] = 0
        else:
            result[i] = (u[i] - np.sum(A[i]) + np.sum(expr)) / denominator
    return result

In [15]:
def ISDnext_tau(lambd, tau, z, A, v, M):
    min_z = np.where(z.T < 0, z.T, 0)
    result = tau.copy()
    delt_A = np.where(A != 0, 1, 0)
    for j in range(result.shape[0]):
        expr = A.T[j]*min_z[j]*M - lambd*delt_A.T[j]
        denominator = np.sum(delt_A.T[j])
        if denominator == 0:
            result[j] = 0
        else:
            result[j] = (v[j] - np.sum(A.T[j]) + np.sum(expr)) / denominator
    return result

In [16]:
def ISDcompute_z(lambd, tau, z, A, M):
    result = copy.deepcopy(z)
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            if A[i][j] == 0:
                result[i][j] = 1
            else:
                var = 1 + ((lambd[i] + tau[j]) / A[i][j])
                if var >= 0:
                    result[i][j] = var
                else:
                    result[i][j] = var / (1 + M)
    return result

In [17]:
M = 100
z = np.ones(A.shape)
lambd = np.zeros(u.shape[0])
tau = np.zeros(v.shape[0])
eps = 1e-8
while 1:
    new_lambd = ISDnext_lambda(lambd, tau, z, A, u, M)
    new_tau = ISDnext_tau(new_lambd, tau, z, A, v, M)
    z = ISDcompute_z(new_lambd, new_tau, z, A, M)
    if np.linalg.norm(new_lambd - lambd) < eps and np.linalg.norm(new_tau - tau) < eps:
        break
    lambd = new_lambd
    tau = new_tau

In [19]:
np.sum(np.round(z*A), axis=0)

array([27695.,   243.,   512., 22399., 57963.,  3500.,  2701.,  5992.,
        4690., 32870., 47939.,  5496.,  7560.,  5966.,  8800., 18643.,
       27720.,  5672., 25233.,  7288.,  6038., 11101.,  8254., 18541.,
        1672.,  7808., 91133., 16983., 83955., 35689., 27406.,  7309.,
        9679., 18859.,  5027., 20051.,  6181.,  4315., 17059., 24502.,
       48631., 21410.,  5965., 69809., 52931., 15011.,  4393.,  8098.,
        6077., 10138., 19099.,  7382., 14314., 74467., 37911., 41938.,
       33790.,  8464.,  5705.,  6052.,   835.,  5120.,   445.,     0.])

In [20]:
np.sum(B, axis=0)

array([27694.,   243.,   512., 22399., 57961.,  3499.,  2703.,  5992.,
        4688., 32869., 47941.,  5497.,  7557.,  5966.,  8799., 18641.,
       27719.,  5671., 25231.,  7289.,  6038., 11103.,  8255., 18540.,
        1673.,  7806., 91131., 16981., 83954., 35689., 27409.,  7308.,
        9679., 18862.,  5027., 20050.,  6183.,  4313., 17057., 24503.,
       48631., 21409.,  5964., 69809., 52932., 15010.,  4394.,  8098.,
        6077., 10141., 19099.,  7382., 14314., 74463., 37910., 41938.,
       33790.,  8464.,  5704.,  6053.,   835.,  5120.,   445.,     0.])

## 2.6 Improved weighted square differences (IWS)

In [21]:
def IWSnext_lambda(lambd, tau, z, A, u, M, devided_A):
    min_z = np.where(z < 0, z, 0)
    result = lambd.copy()      
    for i in range(result.shape[0]):
        expr = A[i]*min_z[i]*M - tau*devided_A[i]
        denominator = np.sum(devided_A[i])           
        if denominator == 0:
            result[i] = 0
        else:
            result[i] = (u[i] - np.sum(A[i]) + np.sum(expr)) / denominator
    return result

In [22]:
def IWSnext_tau(lambd, tau, z, A, v, M, devided_A):
    min_z = np.where(z.T < 0, z.T, 0)
    result = tau.copy()       
    for j in range(result.shape[0]):
        expr = A.T[j]*min_z[j]*M - lambd*devided_A.T[j]
        denominator = np.sum(devided_A.T[j])
        if denominator == 0:
            result[j] = 0
        else:
            result[j] = (v[j] - np.sum(A.T[j]) + np.sum(expr)) / denominator
    return result

In [23]:
def IWScompute_z(lambd, tau, z, A, M):
    result = copy.deepcopy(z)
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            if A[i][j] == 0:
                result[i][j] = 1
            else:
                var = 1 + ((lambd[i] + tau[j])*A[i][j]) / (abs(A[i][j])**3)
                if var >= 0:
                    result[i][j] = var
                else:
                    result[i][j] = var / (1 + M)
    return result

In [24]:
M = 100
z = np.ones(A.shape)
lambd = np.zeros(u.shape[0])
tau = np.zeros(v.shape[0])
eps = 1e-1
devided_A = np.zeros(A.shape)  # It is a matrix which elements are (a_ij)^(-1)
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        devided_A[i][j] = 1/A[i][j] if A[i][j] != 0 else 0
while 1:
    new_lambd = IWSnext_lambda(lambd, tau, z, A, u, M, devided_A)
    new_tau = IWSnext_tau(new_lambd, tau, z, A, v, M, devided_A)
    z = IWScompute_z(new_lambd, new_tau, z, A, M)
    if np.linalg.norm(new_lambd - lambd) < eps and np.linalg.norm(new_tau - tau) < eps:
        break
    lambd = new_lambd
    tau = new_tau

In [26]:
np.sum(np.round(z*A), axis=0)

array([27696.,   243.,   512., 22400., 57961.,  3498.,  2703.,  5991.,
        4688., 32871., 47941.,  5498.,  7558.,  5965.,  8798., 18641.,
       27721.,  5672., 25233.,  7288.,  6039., 11102.,  8255., 18540.,
        1674.,  7806., 91131., 16981., 83954., 35691., 27410.,  7308.,
        9680., 18864.,  5027., 20050.,  6181.,  4312., 17057., 24507.,
       48632., 21408.,  5964., 69809., 52932., 15010.,  4394.,  8097.,
        6076., 10141., 19099.,  7381., 14314., 74465., 37910., 41938.,
       33790.,  8465.,  5704.,  6054.,   837.,  5120.,   445.,     0.])

In [27]:
np.sum(B, axis=0)

array([27694.,   243.,   512., 22399., 57961.,  3499.,  2703.,  5992.,
        4688., 32869., 47941.,  5497.,  7557.,  5966.,  8799., 18641.,
       27719.,  5671., 25231.,  7289.,  6038., 11103.,  8255., 18540.,
        1673.,  7806., 91131., 16981., 83954., 35689., 27409.,  7308.,
        9679., 18862.,  5027., 20050.,  6183.,  4313., 17057., 24503.,
       48631., 21409.,  5964., 69809., 52932., 15010.,  4394.,  8098.,
        6077., 10141., 19099.,  7382., 14314., 74463., 37910., 41938.,
       33790.,  8464.,  5704.,  6053.,   835.,  5120.,   445.,     0.])

In [28]:
methods = {'INS': [next_lambda, next_tau, compute_z], 'ISD': [ISDnext_lambda, ISDnext_tau, ISDcompute_z], 
          'IWS': [IWSnext_lambda, IWSnext_tau, IWScompute_z]}

In [29]:
def predict(previous_year, current_year, method='INS', M=100, eps=1e-8):
    '''This is the main function which taken the previous year data can predict matrix for the next year.
    variables:
    current_year:
        is needed to compute the overall data (summ by rows and columns).
    previous_year:
        data of the previous year
    method:
        It is one of the following strings: "INS", "ISD", "IWS". The chosen method will be used
    M:
        refer to the hyperparameter. For more details refer to the "MainArticle" which is:
        Projection of Supply and Use tables: methods and their empirical assessment
        Working Paper Number: 2
        Authors: Umed Temurshoev, Norihiko Yamano and Colin Webb'''
    if method == 'RAS':
        return RAS_pred(previous_year, current_year, eps)
    A = previous_year
    B = current_year
    u = np.sum(B, axis=1)
    v = np.sum(B, axis=0)
    z = np.ones(A.shape)
    lambd = np.zeros(u.shape[0])
    tau = np.zeros(v.shape[0])
    
    if method == 'IWS':
        devided_A = np.zeros(A.shape)
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                devided_A[i][j] = 1/A[i][j] if A[i][j] != 0 else 0
                
        while 1:
            new_lambd = methods[method][0](lambd, tau, z, A, u, M, devided_A)
            new_tau = methods[method][1](new_lambd, tau, z, A, v, M, devided_A)
            z = methods[method][2](new_lambd, new_tau, z, A, M)
            if np.linalg.norm(new_lambd - lambd) < eps and np.linalg.norm(new_tau - tau) < eps:
                break
            lambd = new_lambd
            tau = new_tau        
        pred_current_year = z*A
        return pred_current_year
                
                
    while 1:
        new_lambd = methods[method][0](lambd, tau, z, A, u, M)
        new_tau = methods[method][1](new_lambd, tau, z, A, v, M)
        z = methods[method][2](new_lambd, new_tau, z, A, M)
        if np.linalg.norm(new_lambd - lambd) < eps and np.linalg.norm(new_tau - tau) < eps:
            break
        lambd = new_lambd
        tau = new_tau
    pred_current_year = z*A
    return pred_current_year

# Metrics

In [30]:
def MAPE(result, true):
    answer = 0
    for i in range(true.shape[0]):
        for j in range(true.shape[1]):
            if true[i][j] != 0:
                answer += abs(result[i][j] - true[i][j]) / abs(true[i][j])
            elif result[i][j] != 0:
                answer += abs(result[i][j] - true[i][j]) / abs(result[i][j])
    return 100*answer / (true.shape[0] * true.shape[1])

In [31]:
def WAPE(result, true):
    answer = 0
    for i in range(true.shape[0]):
        for j in range(true.shape[1]):
            if true[i][j] != 0:
                answer += (abs(true[i][j]) / np.sum(true)) * abs(result[i][j] - true[i][j]) / abs(true[i][j])
            else:
                answer += abs(result[i][j] - true[i][j]) / np.sum(true)
    return answer * 100

In [32]:
def SWAD(result, true):
    return np.sum(np.abs(true*(result - true)))  / np.sum(np.square(true))

## Full circle of predictions

In [34]:
metrics_name = ['MAPE', 'WAPE', 'SWAD']
methods_name = ['IWS', 'INS', 'ISD', 'RAS']
metrics = [MAPE, WAPE, SWAD]

In [35]:
sup10 = np.load('data//sup10.npy')
sup11 = np.load('data//sup11.npy')
sup12 = np.load('data//sup12.npy')
use10 = np.load('data//use10.npy')
use11 = np.load('data//use11.npy')
use12 = np.load('data//use12.npy')

In [36]:
total_results = {i: {method: {metric: 0 for metric in metrics_name} 
                     for method in methods_name} 
                 for i in ['sup11', 'sup12', 'use11', 'use12']}

In [37]:
for method in methods_name:
    if method == 'IWS':
        continue
    M = 10
    eps = 1e-8
    pred_sup11 = predict(sup10, sup11, method=method, M=M, eps=eps)
    pred_sup12 = predict(pred_sup11, sup12, method=method, M=M, eps=eps)
    pred_use11 = predict(use10, use11, method=method, M=M, eps=eps)
    pred_use12 = predict(pred_use11, use12, method=method, M=M, eps=eps)
    for metric_name, metric in zip(metrics_name, metrics):
        total_results['sup11'][method][metric_name] = metric(pred_sup11, sup11)
        total_results['sup12'][method][metric_name] = metric(pred_sup12, sup12)
        total_results['use11'][method][metric_name] = metric(pred_use11, use11)
        total_results['use12'][method][metric_name] = metric(pred_use12, use12)

In [38]:
method = 'IWS'
M = 10
eps = 0.01
pred_sup11 = predict(sup10, sup11, method=method, M=M, eps=eps)
pred_sup12 = predict(np.where(pred_sup11 > 0, pred_sup11, 0), sup12, method=method, M=M, eps=eps)
pred_use11 = predict(use10, use11, method=method, M=M, eps=eps)
pred_use12 = predict(np.where(pred_use11 > 0, pred_use11, 0), use12, method=method, M=M, eps=eps)
for metric_name, metric in zip(metrics_name, metrics):
    total_results['sup11'][method][metric_name] = metric(pred_sup11, sup11)
    total_results['sup12'][method][metric_name] = metric(pred_sup12, sup12)
    total_results['use11'][method][metric_name] = metric(pred_use11, use11)
    total_results['use12'][method][metric_name] = metric(pred_use12, use12)

In [410]:
print('Рузальтаты при M=100.000')
for instance in ['sup11', 'sup12', 'use11', 'use12']:
    print('-----------------------'+instance+'--------------------')
    print(pd.DataFrame(total_results[instance]))
    print('------------------------------------------------')
    print()

Рузальтаты при M=100.000
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  319.211952  7.583174  157.425026  81.943565
WAPE   13.684159  1.700187   10.664708   5.827249
SWAD    0.052362  0.004038    0.046185   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  337.659387  9.755207  253.139045  83.090178
WAPE   18.646112  2.381380   14.243923   5.119052
SWAD    0.070047  0.004930    0.059324   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  331.943745  11.993685  156.975806  33.427618
WAPE   21.394158   5.013336   17.097695   9.515708
SWAD    0.133709   0.021530    0.125398   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        I

In [413]:
print('Рузальтаты при M=10.000')
for instance in ['sup11', 'sup12', 'use11', 'use12']:
    print('-----------------------'+instance+'--------------------')
    print(pd.DataFrame(total_results[instance]))
    print('------------------------------------------------')
    print()

Рузальтаты при M=10.000
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  322.246585  7.583174  157.433136  81.943565
WAPE   13.699382  1.700187   10.664767   5.827249
SWAD    0.052370  0.004038    0.046185   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  348.630454  9.755207  253.147606  83.090178
WAPE   18.694534  2.381380   14.243971   5.119052
SWAD    0.070136  0.004930    0.059324   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  334.347443  11.993685  156.994151  33.427618
WAPE   21.411596   5.013336   17.097805   9.515708
SWAD    0.133708   0.021530    0.125399   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        IN

In [416]:
print('Рузальтаты при M=1.000')
for instance in ['sup11', 'sup12', 'use11', 'use12']:
    print('-----------------------'+instance+'--------------------')
    print(pd.DataFrame(total_results[instance]))
    print('------------------------------------------------')
    print()

Рузальтаты при M=1.000
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  349.092392  7.583174  157.513699  81.943565
WAPE   13.807266  1.700187   10.665363   5.827249
SWAD    0.052405  0.004038    0.046185   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  360.203661  9.755207  253.227123  83.090178
WAPE   18.706378  2.381380   14.244367   5.119052
SWAD    0.070169  0.004930    0.059328   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  355.488175  11.993685  157.176774  33.427618
WAPE   21.567331   5.013336   17.098901   9.515708
SWAD    0.133695   0.021530    0.125400   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        INS

In [419]:
print('Рузальтаты при M=100')
for instance in ['sup11', 'sup12', 'use11', 'use12']:
    print('-----------------------'+instance+'--------------------')
    print(pd.DataFrame(total_results[instance]))
    print('------------------------------------------------')
    print()

Рузальтаты при M=100
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  473.186929  7.583174  158.291903  81.943565
WAPE   14.260326  1.700187   10.671134   5.827249
SWAD    0.052417  0.004038    0.046189   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  413.803444  9.755207  254.167487  83.090178
WAPE   19.330434  2.381380   14.248677   5.119052
SWAD    0.070235  0.004930    0.059360   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  463.498420  11.993685  158.932695  33.427618
WAPE   22.285158   5.013336   17.109343   9.515708
SWAD    0.133641   0.021530    0.125409   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        INS  

In [39]:
print('Рузальтаты при M=10')
for instance in ['sup11', 'sup12', 'use11', 'use12']:
    print('-----------------------'+instance+'--------------------')
    print(pd.DataFrame(total_results[instance]))
    print('------------------------------------------------')
    print()

Рузальтаты при M=10
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  683.883327  7.583174  164.145255  81.943565
WAPE   15.155620  1.700187   10.716287   5.827249
SWAD    0.052401  0.004038    0.046222   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  547.439884  9.755207  262.674272  83.090178
WAPE   20.472053  2.381380   14.281199   5.119052
SWAD    0.070181  0.004930    0.059595   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  693.936618  11.993685  171.258934  33.427618
WAPE   23.836943   5.013336   17.186104   9.515708
SWAD    0.133579   0.021530    0.125476   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        INS   

In [44]:
print('Рузальтаты при M=0')
for instance in ['sup11', 'sup12', 'use11', 'use12']:
    print('-----------------------'+instance+'--------------------')
    print(pd.DataFrame(total_results[instance]))
    print('------------------------------------------------')
    print()

Рузальтаты при M=0
-----------------------sup11--------------------
             IWS       INS         ISD        RAS
MAPE  852.871567  7.583174  191.204137  81.943565
WAPE   16.025143  1.700187   10.940347   5.827249
SWAD    0.052394  0.004038    0.046383   0.049272
------------------------------------------------

-----------------------sup12--------------------
             IWS       INS         ISD        RAS
MAPE  729.721125  9.755207  306.896772  83.090178
WAPE   22.099099  2.381380   14.609489   5.119052
SWAD    0.070144  0.004930    0.060405   0.037626
------------------------------------------------

-----------------------use11--------------------
             IWS        INS         ISD        RAS
MAPE  941.412129  11.993685  217.577982  33.427618
WAPE   25.348919   5.013336   17.466787   9.515708
SWAD    0.133537   0.021530    0.125710   0.075513
------------------------------------------------

-----------------------use12--------------------
             IWS        INS    